In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install moviepy streamlit pyngrok transformers torch

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-4it1ufl9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-4it1ufl9
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
%%writefile app.py
import streamlit as st
import os
import tempfile
import whisper
from moviepy.editor import VideoFileClip
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from pyngrok import ngrok
# Load pre-trained BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

def convert_to_audio(file_path):
    name, ext = os.path.splitext(file_path)
    if ext.lower() in ['.mp4', '.avi', '.mov', '.mkv']:  # Video formats
        audio_path = f"{name}.mp3"
        video = VideoFileClip(file_path)
        video.audio.write_audiofile(audio_path)
        return audio_path
    return file_path  # Already an audio file

def transcribe_audio(audio_path):
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["text"]

def generate_response(prompt, max_length=250):
    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)
    output_ids = model.generate(inputs, max_length=max_length, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

def generate_insights(transcription, insight_type="general", num_insights=5, content_type="interview"):
    extraction_prompts = {
        "general": f"Extract the main points and overall themes from this {content_type}:\n\n{transcription}\n\nMain points:",
        "technical": f"Extract all mentions of technologies, technical processes, and innovations from this {content_type}:\n\n{transcription}\n\nTechnical points:",
        "business": f"Extract all mentions of business strategies, market positioning, product features, and future plans from this {content_type}:\n\n{transcription}\n\nBusiness points:"
    }

    extracted_info = generate_response(extraction_prompts[insight_type], max_length=300)

    insight_prompts = {
        "general": f"Based on these main points from a {content_type}, provide {num_insights} key general insights:\n\n{extracted_info}\n\nGeneral Insights:",
        "technical": f"Based on these technical points from a {content_type}, provide {num_insights} key technical insights:\n\n{extracted_info}\n\nTechnical Insights:",
        "business": f"Based on these business points from a {content_type}, provide {num_insights} key business insights:\n\n{extracted_info}\n\nBusiness Insights:"
    }

    insights = generate_response(insight_prompts[insight_type], max_length=300)

    insights_list = insights.split(". ")
    formatted_insights = f"{insight_type.capitalize()} Insights for the {content_type}:\n\n"
    for i, insight in enumerate(insights_list[:num_insights], 1):
        formatted_insights += f"{i}. {insight.strip()}\n"

    return formatted_insights

def answer_followup(question, transcription):
    prompt = f"Based on the following transcription, answer this question: {question}\n\nTranscription: {transcription}\n\nAnswer:"
    return generate_response(prompt, max_length=200)

def main():
    st.set_page_config(page_title="Interview Insight Generator", page_icon="🎙️", layout="wide")
    st.title("🎙️ Interview Insight Generator")
    st.markdown("---")
    st.sidebar.header("About")
    st.sidebar.info("This app transcribes audio and video interviews, generates insights, and answers follow-up questions using AI.")

    col1, col2 = st.columns(2)

    insights = None
    transcription = ""  # Initialize transcription variable

    with col1:
        st.subheader("Upload Interview")
        uploaded_file = st.file_uploader("Choose an audio or video file", type=['wav', 'mp3', 'm4a', 'mp4', 'avi', 'mov', 'mkv'])

        if uploaded_file is not None:
            with st.spinner('Transcribing...'):
                try:
                    with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(uploaded_file.name)[1]) as tmp_file:
                        tmp_file.write(uploaded_file.getvalue())
                        tmp_file_path = tmp_file.name

                    audio_path = convert_to_audio(tmp_file_path)
                    transcription = transcribe_audio(audio_path)

                    if transcription:
                        st.success("Transcription complete!")
                        with st.expander("View Transcription"):
                            st.text_area("", transcription, height=200)
                    else:
                        st.error("Transcription failed or returned empty result.")

                    os.unlink(tmp_file_path)
                    if audio_path != tmp_file_path:
                        os.unlink(audio_path)

                except Exception as e:
                    st.error(f"An error occurred during processing: {str(e)}")

            st.subheader("Insight Generation Options")
            insight_type = st.selectbox(
                "Select insight type",
                ["general", "technical", "business"]
            )
            num_insights = st.slider("Number of insights", min_value=3, max_value=10, value=5)
            content_type = st.text_input("Content type (e.g., interview, presentation)", "interview")

            if st.button("Generate Insights"):
                with st.spinner('Generating insights...'):
                    insights = generate_insights(
                        transcription,
                        insight_type=insight_type,
                        num_insights=num_insights,
                        content_type=content_type
                    )
                st.success("Insights generated!")

    with col2:
        if insights:
            st.subheader("AI-Generated Insights")
            st.text_area("", insights, height=300)

        if transcription:  # Only show follow-up section if there's a transcription
            st.subheader("Ask Follow-up Questions")
            question = st.text_input("Enter your question here")
            if question:
                with st.spinner('Generating answer...'):
                    answer = answer_followup(question, transcription)
                st.info(answer)

    st.markdown("---")
    st.markdown("Created by Ama Annor and Claude Quartey")

if __name__ == '__main__':
    main()

Overwriting app.py


In [3]:
!ngrok authtoken 2kG58KDWV2CW3P7TwD5foJuvClZ_4JUb7T4Jivc734zsgRcch

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
import os
os.environ["NGROK_AUTHTOKEN"] = "2kG58KDWV2CW3P7TwD5foJuvClZ_4JUb7T4Jivc734zsgRcch"

In [5]:
!killall ngrok # Kill any existing ngrok processes
!streamlit run app.py &>/dev/null&

# Setup ngrok tunnel
from pyngrok import ngrok
public_url = ngrok.connect(addr='8501')
print(f"Public URL: {public_url}")

ngrok: no process found
Public URL: NgrokTunnel: "https://3f87-34-74-253-160.ngrok-free.app" -> "http://localhost:8501"
